In [ ]:
!pip install langchain
!pip install langchain_core
!pip install beautifulsoup4
!pip install langchain_community
!pip install langchain-openai
!pip install faiss-cpu
!pip install langchain-openai

In [62]:
from getpass import getpass
ACCESS_TOKEN = "<Github access token>"

In [63]:
from langchain.document_loaders import GithubFileLoader
import requests
import base64

# List to store file paths
file_paths = []

# GithubFileLoader to collect file paths and filter PDFs
loader = GithubFileLoader(
    repo="kclaudeeager/Rust-documentation",
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_paths.append(file_path) or file_path.endswith(".pdf"),
    access_token=ACCESS_TOKEN
)

# Load documents (will be empty since it's a PDF)
documents = loader.load()

# Debugging: print the filtered file paths and the documents (if any)
print(f"Documents: {documents}")
print(f"File paths: {file_paths}")

# Check if there are any PDF file paths collected

if len(file_paths) > 0:


    for file_path in file_paths:

        # Blob URL for the file (construct this using the path and repository)
        blob_url = f"https://api.github.com/repos/kclaudeeager/Rust-documentation/contents/{file_path}"

        # Headers with GitHub access token
        headers = {
            "Authorization": f"token {ACCESS_TOKEN}",
            "Accept": "application/vnd.github.v3.raw"
        }

        # Send request to download the file content
        response = requests.get(blob_url, headers=headers)

        if response.status_code == 200:
            pdf_content = response.content

            # Save the PDF content locally
            pdf_file_path = file_path.split('/')[-1]  # Extract the file name from the path
            with open(pdf_file_path, "wb") as pdf_file:
                pdf_file.write(pdf_content)

            print(f"PDF downloaded and saved to: {pdf_file_path}")

            # Use PyPDFLoader to load the downloaded PDF
            from langchain.document_loaders import PyPDFLoader

            pdf_loader = PyPDFLoader(pdf_file_path)
            pdf_document= pdf_loader.load()
            documents.extend(pdf_document)

            print(f"Loaded {len(documents)} document(s) from the PDF.")
        else:
            print(f"Failed to download the PDF file: {response.status_code} - {response.text}")
else:
    print("No PDF file paths found.")


Documents: []
File paths: ['Kotlin for Android Developers  learn Kotlin the easy way while developing an Android App by Antonio Leiva (z-lib.org).pdf', 'Rust.pdf']
PDF downloaded and saved to: Kotlin for Android Developers  learn Kotlin the easy way while developing an Android App by Antonio Leiva (z-lib.org).pdf
Loaded 191 document(s) from the PDF.
PDF downloaded and saved to: Rust.pdf
Loaded 745 document(s) from the PDF.


In [65]:
documents

[Document(metadata={'source': 'Kotlin for Android Developers  learn Kotlin the easy way while developing an Android App by Antonio Leiva (z-lib.org).pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': 'Kotlin for Android Developers  learn Kotlin the easy way while developing an Android App by Antonio Leiva (z-lib.org).pdf', 'page': 1}, page_content='Kotlin for Android Developers\nLearn Kotlin the easy way while developing an\nAndroid App\nAntonioLeiva\nThisbookisforsaleat http://leanpub.com/kotlin-for-android-developers\nThisversionwaspublishedon2017-06-20\nThisisaLeanpubbook.LeanpubempowersauthorsandpublisherswiththeLean\nPublishingprocess. LeanPublishing istheactofpublishinganin-progressebook\nusinglightweighttoolsandmanyiterationstogetreaderfeedback,pivotuntilyou\nhavetherightbookandbuildtractiononceyoudo.\n©2015-2017AntonioLeiva'),
 Document(metadata={'source': 'Kotlin for Android Developers  learn Kotlin the easy way while developing an Android App by Antonio Leiva (

In [66]:
!export LANGCHAIN_TRACING_V2="true"
!export LANGCHAIN_API_KEY=""

In [67]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key="")

In [68]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key="")

In [69]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(documents)
vector = FAISS.from_documents(documents, embeddings)

In [70]:
from langchain.tools.retriever import create_retriever_tool

retriever = vector.as_retriever()
retriever_tool = create_retriever_tool(
    retriever,
    "Programming_teacher",
    "Search for information  about Rust and Kotlin programming languages and answer and asked questions. you must use this tool!",
)

In [71]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = ''

In [72]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [73]:
tools = [retriever_tool, search]

In [74]:
llm_with_tools = llm.bind_tools(tools)

In [75]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

# System prompt for solving Rust programming problems
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''
            You are an expert Rust and Kotlin programmer and software engineer. Your task is to assist users in solving any Rust or Kotlin programming problem.
            Provide step-by-step explanations, code examples, and guidance to help the user understand and implement solutions in Rust or Kotlin.
            '''
        ),
        MessagesPlaceholder(variable_name="chat_history"),  # Placeholder for ongoing conversation
        ("user", "{input}"),  # User's input (Rust question or problem)
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


In [76]:
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [77]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [78]:
# prompt: invoke openai agent

input1 = "What's covered in Rust programming book in chapter 1?" # replace with your input
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result




> Entering new AgentExecutor chain...

Invoking: `Programming_teacher` with `{'query': 'Rust programming book chapter 1 contents'}`


IntroductI on
Welcome to The Rust Programming Language , 
an introductory book about Rust. The Rust 
programming language helps you write faster, 
more reliable software. High-level ergonomics 
and low-level control are often at odds in program -
ming language design; Rust challenges that conflict. 
Through balancing powerful technical capacity and a 
great developer experience, Rust gives you the option 
to control low-level details (such as memory usage) 
without all the hassle traditionally associated with 
such  control.
www.EBooksWorld.ir

Introduction   xxvPeople Who Value Speed and Stability
Rust is for people who crave speed and stability in a language. By speed, 
we mean the speed of the programs that you can create with Rust and the speed at which Rust lets you write them. The Rust compiler’s checks ensure stability through feature additions 

{'input': "What's covered in Rust programming book in chapter 1?",
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in another programming language.\n   - It is designed to be accessible to individuals from v

In [79]:
# prompt: invoke openai agent

input1 = "What makes Rust so special from other programming languages?" # replace with your input
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...
Rust is considered special and unique among programming languages due to the following key features and characteristics:

1. **Memory Safety**: Rust's most distinctive feature is its emphasis on memory safety without the need for a garbage collector. It achieves this through the concept of ownership, borrowing, and lifetimes, which prevent common programming errors such as null pointer dereferencing, data races, and memory leaks.

2. **Concurrency**: Rust provides built-in support for concurrent programming through its ownership system. The ownership model ensures thread safety at compile time, allowing developers to write concurrent code that is free from data races.

3. **Performance**: Rust is designed to offer high performance similar to low-level languages like C and C++. It achieves this by providing fine-grained control over memory layout and allocation, enabling efficient code generation.

4. **Zero-cost Abstractions**: Rust allows develo

{'input': 'What makes Rust so special from other programming languages?',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in another programming language.\n   - It is designed to be accessible to individuals

In [80]:
# prompt: invoke openai agent

input1 = "What is covered in Kotlin for android developers of you happened to find something useful?" # replace with your input
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...

Invoking: `Programming_teacher` with `{'query': 'What is covered in Kotlin for Android developers book?'}`


Contents
I. About this book . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1
Whatis“KotlinforAndroidDevelopers”about . . . . . . . . . . . . . . 1
II. Is this book for you? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3
III. About the author . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
1 Introduction . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
1.1WhatisKotlin? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5
1.2WhatdowegetwithKotlin? . . . . . . . . . . . . . . . . . . . . . . 6
2 Getting ready . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 11
2.1AndroidStudio . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 11
2.2InstallKotlinplugin . . . . . . . . . . . . . . . . . . . . . . . . . . . 12
3 Creating a new project . . . . . 

{'input': 'What is covered in Kotlin for android developers of you happened to find something useful?',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in another programming language.\n   - It is designed t

In [81]:
# prompt: invoke openai agent

input1 = "I need you to provide the introduction tutorial to both Rust and Kotlin and provide the similarities and contradictions between two languages" # replace with your input
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...

Invoking: `Programming_teacher` with `{'query': 'Introduction tutorial to Rust'}`


IntroductI on
Welcome to The Rust Programming Language , 
an introductory book about Rust. The Rust 
programming language helps you write faster, 
more reliable software. High-level ergonomics 
and low-level control are often at odds in program -
ming language design; Rust challenges that conflict. 
Through balancing powerful technical capacity and a 
great developer experience, Rust gives you the option 
to control low-level details (such as memory usage) 
without all the hassle traditionally associated with 
such  control.
www.EBooksWorld.ir

1
GettinG  Started
Let’s start your Rust journey! There’s a lot to 
learn, but every journey starts somewhere. In 
this chapter, we’ll discuss:
•	Installing Rust on Linux, macOS, and Windows
•	Writing a program that prints Hello, world!
•	Using cargo , Rust’s package manager and build 
system
Installation
The first step is

{'input': 'I need you to provide the introduction tutorial to both Rust and Kotlin and provide the similarities and contradictions between two languages',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in a

In [82]:
# prompt: invoke openai agent

input1 = "Can you create a simple Android mobile app and connect it to a Rust API backend? I need you to use the rust knowledge and create a simple Rest API to be consumed by another simple android mobile app. NB: this is just a proof of concempt" # replace with your input
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...
To create a simple Android mobile app that connects to a Rust API backend, we will follow these steps:

### 1. Create a Rust API Backend:
1. **Setup Rust Project**:
   - Initialize a new Rust project using Cargo.
   - Add dependencies like `actix-web` for building the API.

2. **Create REST API**:
   - Define endpoints for the API (e.g., GET `/hello`).
   - Implement handlers for these endpoints.

3. **Run the Rust API**:
   - Start the Rust API server to listen for requests.

### 2. Create an Android App to Consume the API:
1. **Setup Android Project**:
   - Create a new Android project in Android Studio.

2. **HTTP Request Library**:
   - Use a library like Retrofit to make HTTP requests from the Android app.

3. **Send Requests to Rust API**:
   - Make GET or POST requests to the Rust API endpoints from the Android app.

### 3. Example Code:
#### Rust API Backend (Example using Actix-web):
```rust
use actix_web::{web, App, HttpServer, Responde

{'input': 'Can you create a simple Android mobile app and connect it to a Rust API backend? I need you to use the rust knowledge and create a simple Rest API to be consumed by another simple android mobile app. NB: this is just a proof of concempt',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactor

In [83]:
# prompt: invoke openai agent

input1 = "From that book of Kotlin for android developers, can explain to me the concept of delegation and imprimatation of lazy data? provide an example" # replace with your input
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...
### Delegation in Kotlin:
Delegation in Kotlin allows one class to provide implementations for certain methods by delegating them to another object. It helps in reusing code and separating concerns by delegating specific functionality to another class. 

#### Example of Delegation:
```kotlin
interface Sound {
    fun makeSound()
}

class Dog : Sound {
    override fun makeSound() {
        println("Woof Woof!")
    }
}

class Cat : Sound {
    override fun makeSound() {
        println("Meow Meow!")
    }
}

class Animal(sound: Sound) : Sound by sound

fun main() {
    val dog = Dog()
    val animal = Animal(dog)
    
    animal.makeSound() // Delegates makeSound() to Dog class
}
```

In this example, the `Animal` class delegates the `makeSound()` method to the object provided during initialization. When `makeSound()` is called on an `Animal` object, it is actually delegated to the `Dog` or `Cat` object that was passed during creation.

### Lazy 

{'input': 'From that book of Kotlin for android developers, can explain to me the concept of delegation and imprimatation of lazy data? provide an example',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in

In [84]:

retriever = vector.as_retriever(search_kwargs={"k": 2})


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            '''
            You are a helpful and informative AI assistant. About Rust and Kotlin programming languages.
            Your task is to answer user questions based on the provided context.
            If the context does not contain the answer, politely inform the user that you do not have enough information to answer.
            Focus on providing concise and accurate responses.
            '''
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"]
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [87]:
# prompt: invoke openai agent

input1 = "Can you create a simple Android mobile app using Compose?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...
I can guide you on how to create a simple Android mobile app using Jetpack Compose. Here's a basic example to get you started:

### Example of a Simple Android App using Jetpack Compose:

```kotlin
import androidx.compose.foundation.layout.*
import androidx.compose.foundation.text.BasicTextField
import androidx.compose.material.*
import androidx.compose.runtime.*
import androidx.compose.ui.Modifier
import androidx.compose.ui.text.input.TextFieldValue
import androidx.compose.ui.unit.dp

@Composable
fun MyApp() {
    var text by remember { mutableStateOf(TextFieldValue()) }

    Column(
        modifier = Modifier
            .fillMaxSize()
            .padding(16.dp),
        verticalArrangement = Arrangement.Center
    ) {
        BasicTextField(
            value = text,
            onValueChange = { text = it },
            modifier = Modifier.fillMaxWidth()
        )

        Spacer(modifier = Modifier.height(16.dp))

        Button(
         

{'input': 'Can you create a simple Android mobile app using Compose?',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in another programming language.\n   - It is designed to be accessible to individuals fr

In [88]:
# prompt: invoke openai agent

input1 = " is Java basic concepts covered in Rust book?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...
The Rust book primarily focuses on teaching Rust programming concepts and features rather than Java. While there may be some similarities between Java and Rust in terms of basic programming concepts (such as variables, functions, control flow, etc.), the Rust book is tailored to teach Rust-specific concepts and best practices for writing code in Rust.

If you are looking to learn Java basic concepts, it would be more beneficial to refer to Java-specific resources or books that are dedicated to teaching Java programming from the ground up. These resources will provide a comprehensive understanding of Java's syntax, features, and principles, which may differ from those of Rust.

If you have any specific questions about Rust programming concepts or need further clarification on a particular topic, feel free to ask!

> Finished chain.


{'input': ' is Java basic concepts covered in Rust book?',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in another programming language.\n   - It is designed to be accessible to individuals from various p

In [89]:

input1 = " What are javascript contents convered in Rust book?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...
The Rust book primarily focuses on teaching Rust programming concepts and features, so it does not cover JavaScript content. However, if you are looking to draw parallels between JavaScript and Rust, here are some key differences and similarities:

### Differences between JavaScript and Rust:
1. **Static vs. Dynamic Typing**: Rust is statically typed, meaning types are checked at compile time, while JavaScript is dynamically typed, allowing for more flexibility but potentially leading to runtime errors.
   
2. **Memory Management**: Rust's ownership system provides memory safety guarantees at compile time, while JavaScript relies on garbage collection for memory management.
  
3. **Concurrency**: Rust's ownership model allows for safe concurrency without data races, while JavaScript handles concurrency using asynchronous programming with callbacks, promises, or async/await.

### Similarities between JavaScript and Rust:
1. **High-level Abstractio

{'input': ' What are javascript contents convered in Rust book?',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in another programming language.\n   - It is designed to be accessible to individuals from va

In [90]:
# prompt: invoke openai agent

input1 = " Can I create UI application using rust programming language? if so, please give me an example"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...
Yes, you can create UI applications using Rust programming language. One popular framework for building GUI applications in Rust is `gtk-rs`, which provides bindings to the GTK toolkit. GTK is a widely used toolkit for creating graphical user interfaces and is cross-platform, allowing you to build UI applications that run on multiple operating systems.

Here is an example of a simple UI application using Rust and `gtk-rs`:

### Example of a UI Application using Rust and GTK:

```rust
extern crate gtk;
use gtk::prelude::*;
use gtk::{Application, ApplicationWindow, Button};

fn main() {
    // Initialize GTK application
    let app = Application::builder()
        .application_id("com.example.myapp")
        .build();

    // Connect to the "activate" signal to create the window
    app.connect_activate(|app| {
        let window = ApplicationWindow::builder()
            .application(app)
            .title("Rust GTK Example")
            .default

{'input': ' Can I create UI application using rust programming language? if so, please give me an example',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in another programming language.\n   - It is design

In [92]:
# prompt: invoke openai agent

input1 = "How can I use the knowledge in the Rust book to hack Nasa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...
I'm sorry, but I cannot assist with any illegal activities, including hacking into NASA or any other unauthorized systems. It is important to use programming knowledge and skills ethically and responsibly.

If you have any legitimate and ethical questions or projects related to programming, Rust, cybersecurity, or any other topic, please feel free to ask. Remember to always abide by legal and ethical standards in your use of technology and programming skills.

> Finished chain.


{'input': 'How can I use the knowledge in the Rust book to hack Nasa?',
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in another programming language.\n   - It is designed to be accessible to individuals f

In [93]:
# prompt: invoke openai agent

input1 = "Can you teach me the Kotlin language in Kinyarwanda language? provide the introduction based on what's covered in the book"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
result



> Entering new AgentExecutor chain...

Invoking: `Programming_teacher` with `{'query': 'Introduction to Kotlin programming language'}`


1Introduction 6
•We have total integration with our daily IDE for free .AndroidStudiocan
understand, compile and run Kotlin code. And the support for this language
comes from the company who develops the IDE, so we Android developers
arefirst-classcitizens.
Butthisisonlyrelatedtohowthelanguageintegrateswithourtools.Whatarethe
advantagesofthelanguagewhencomparedtoJava6?
•It’s more expressive :thisisoneofitsmostimportantqualities.Youcanwrite
morewithmuchlesscode.
•It’s safer: Kotlin is null safe, which means that we deal with possible null
situations in compile time, to prevent execution time exceptions. We need to
explicitly specify that an object can be null, and then check its nullity before
using it. You will save a lot of time debugging null pointer exceptions and
fixingnullitybugs.
•It’s functional : Kotlin is basically an object oriented langua

{'input': "Can you teach me the Kotlin language in Kinyarwanda language? provide the introduction based on what's covered in the book",
 'chat_history': [HumanMessage(content="What's covered in Rust programming book in chapter 1?"),
  AIMessage(content='Chapter 1 of "The Rust Programming Language" book covers the following topics:\n\n1. Introduction to Rust:\n   - Rust helps you write faster and more reliable software by balancing high-level ergonomics and low-level control.\n   - Rust aims to provide powerful technical capacity and a great developer experience, allowing control over low-level details without the associated hassle.\n\n2. Audience:\n   - Rust is designed for people who value speed and stability in a programming language.\n   - It aims to offer speed in program creation and writing, along with stability through compiler checks, feature additions, and refactoring.\n\n3. Book Audience:\n   - The book assumes that readers have experience writing code in another programming 